In [3]:
# ================================================================
#  SET-ME-FIRST  ▸ auto-locate CSVs & global knobs
# ================================================================
from pathlib import Path
import pandas as pd, sys

# ----------------------------------------------------------------
# 1) Find repo root  (= directory where notebook was started)
# ----------------------------------------------------------------
REPO_ROOT = Path.cwd()

# ----------------------------------------------------------------
# 2) Candidate locations for the CSVs, in search order
# ----------------------------------------------------------------
candidates = [
    REPO_ROOT / "data",
    REPO_ROOT,                                  # flat files next to notebook
    REPO_ROOT / "python_testing1",              # your original folder
    REPO_ROOT.parent / "data",
    REPO_ROOT.parent / "python_testing1",
]

# give up after walking two levels deep
for p in REPO_ROOT.rglob("*"):
    if p.is_dir() and p.relative_to(REPO_ROOT).parts[:2] == p.parts[-2:]:
        candidates.append(p)

# ----------------------------------------------------------------
# 3) Pick first folder that has BOTH orderbook files
# ----------------------------------------------------------------
bnc_path = htx_path = None
for folder in candidates:
    b = folder / "orderbook_bnc.csv"
    h = folder / "orderbook_hb.csv"
    if b.exists() and h.exists():
        bnc_path, htx_path = b, h
        break

if not bnc_path:
    print("❌ Could not find order-book CSVs. "
          "Put them in ./data/ or ./python_testing1/")
    sys.exit(1)

DATA_DIR = bnc_path.parent
print(f"✓ Using data in: {DATA_DIR}")

# ----------------------------------------------------------------
# 4) Universal knobs users may tweak
# ----------------------------------------------------------------
NOTIONAL_USD      = 100_000.0
MAX_LVL           = 10
TRIGGER_PCT       = 0.002        # 0.002 %

MAKER_HTX_BP , TAKER_BNC_BP = 2.24 , 2.80
MAKER_BNC_BP , TAKER_HTX_BP = 1.90 , 3.18

WITHDRAW_FEE_LNK  = 0.11
TRANSFER_TH_LINK  = 10_000


✓ Using data in: c:\Users\jklee\Downloads\python_testing1


In [4]:
import numpy as np, pandas as pd

# ─── master knobs ────────────────────────────────────────────────
NOTIONAL_USD = 100_000.0
MAX_LVL      = 10
TRIGGER_PCT  = 0.002        # 0.002 %

# fee schedule (bp)
MAKER_HTX_BP , TAKER_BNC_BP = 2.24 , 2.80
MAKER_BNC_BP , TAKER_HTX_BP = 1.90 , 3.18

WITHDRAW_FEE_LNK = 0.11
TRANSFER_TH_LINK = 10_000

# ─── helper -------------------------------------------------------
def vwap_partial(row, px_pre, vol_pre, usd_target, levels=MAX_LVL):
    remain, cost, qty = usd_target, 0.0, 0.0
    for lvl in range(1, levels + 1):
        px, vol = row[f"{px_pre}{lvl}"], row[f"{vol_pre}{lvl}"]
        take = min(px * vol, remain)
        cost += take; qty += take / px; remain -= take
        if remain <= 0: break
    return (cost / qty if qty else np.nan), cost, qty

# ─── load books ---------------------------------------------------
bnc = (pd.read_csv(bnc_path, parse_dates=["timestamp"])
         .set_index("timestamp").add_prefix("BNC_"))
htx = (pd.read_csv(htx_path, parse_dates=["timestamp"])
         .set_index("timestamp").add_prefix("HTX_"))
df  = (pd.merge(bnc, htx, left_index=True, right_index=True, how="outer")
         .sort_index().ffill().dropna())

# ─── allocate output columns -------------------------------------
cols = [
    # SHBB
    "VWAP_SH_M","VWAP_SH_T","USD_SH","Q_SH",
    # BBSH
    "VWAP_BB_M","VWAP_BB_T","USD_BB","Q_BB",
    # SBBH
    "VWAP_SB_M","VWAP_SB_T","USD_SB","Q_SB",
    # BHSB
    "VWAP_HS_M","VWAP_HS_T","USD_HS","Q_HS"]
df[cols] = np.nan

for idx, row in df.iterrows():

    # ── SHBB  (HTX ask SELL  →  BNC ask BUY) ─────────────────────
    v_m, usd_m, q = vwap_partial(row, "HTX_a", "HTX_av", NOTIONAL_USD)
    if not np.isnan(v_m):
        v_t, _, _ = vwap_partial(row, "BNC_a", "BNC_av", usd_m)
        df.loc[idx, ["VWAP_SH_M","VWAP_SH_T","USD_SH","Q_SH"]] = v_m, v_t, usd_m, q

    # ── BBSH  (BNC bid BUY   →  HTX bid SELL) ────────────────────
    v_m, usd_m, q = vwap_partial(row, "BNC_b", "BNC_bv", NOTIONAL_USD)
    if not np.isnan(v_m):
        v_t, _, _ = vwap_partial(row, "HTX_b", "HTX_bv", usd_m)
        df.loc[idx, ["VWAP_BB_M","VWAP_BB_T","USD_BB","Q_BB"]] = v_m, v_t, usd_m, q

    # ── SBBH  (BNC ask SELL  →  HTX ask BUY) ─────────────────────
    v_m, usd_m, q = vwap_partial(row, "BNC_a", "BNC_av", NOTIONAL_USD)
    if not np.isnan(v_m):
        v_t, _, _ = vwap_partial(row, "HTX_a", "HTX_av", usd_m)
        df.loc[idx, ["VWAP_SB_M","VWAP_SB_T","USD_SB","Q_SB"]] = v_m, v_t, usd_m, q

    # ── BHSB  (HTX bid BUY   →  BNC bid SELL) ────────────────────
    v_m, usd_m, q = vwap_partial(row, "HTX_b", "HTX_bv", NOTIONAL_USD)
    if not np.isnan(v_m):
        v_t, _, _ = vwap_partial(row, "BNC_b", "BNC_bv", usd_m)
        df.loc[idx, ["VWAP_HS_M","VWAP_HS_T","USD_HS","Q_HS"]] = v_m, v_t, usd_m, q

# ─── edge % calculations -----------------------------------------
df["edge_SH"] = (
    (df["VWAP_SH_M"] - df["VWAP_SH_T"]
     - df["VWAP_SH_M"]*MAKER_HTX_BP/10_000
     - df["VWAP_SH_T"]*TAKER_BNC_BP/10_000)
    / df["VWAP_SH_T"] * 100)

df["edge_BB"] = (
    (df["VWAP_BB_T"] - df["VWAP_BB_M"]
     - df["VWAP_BB_M"]*MAKER_BNC_BP/10_000
     - df["VWAP_BB_T"]*TAKER_HTX_BP/10_000)
    / df["VWAP_BB_M"] * 100)

df["edge_SB"] = (
    (df["VWAP_SB_M"] - df["VWAP_SB_T"]
     - df["VWAP_SB_M"]*MAKER_BNC_BP/10_000
     - df["VWAP_SB_T"]*TAKER_HTX_BP/10_000)
    / df["VWAP_SB_T"] * 100)

df["edge_HS"] = (
    (df["VWAP_HS_T"] - df["VWAP_HS_M"]
     - df["VWAP_HS_M"]*MAKER_HTX_BP/10_000
     - df["VWAP_HS_T"]*TAKER_BNC_BP/10_000)
    / df["VWAP_HS_M"] * 100)


In [5]:
import plotly.graph_objects as go, plotly.io as pio
from plotly.subplots import make_subplots
pio.templates.default = "plotly_dark"

# ─── create a 2×1 subplot grid ────────────────────────────────────
fig = make_subplots(rows=2, cols=1,
                    shared_xaxes=True,
                    vertical_spacing=0.03,
                    subplot_titles=(
                        "Maker-SELL legs  ➜  SHBB, SBBH",
                        "Maker-BUY legs   ➜  BBSH, BHSB"))

# ── Row-1: maker limits are SELLS (edges expressed vs. BUY price) ─
fig.add_scatter(x=df.index, y=df["edge_SH"],
                mode="lines", name="SHBB  (Sell HTX / Buy BNC)",
                line=dict(width=1), legendgroup="row1", row=1, col=1)

fig.add_scatter(x=df.index, y=df["edge_SB"],
                mode="lines", name="SBBH  (Sell BNC / Buy HTX)",
                line=dict(width=1), legendgroup="row1", row=1, col=1)

# trigger line (only need once because x-axis is shared)
fig.add_hline(y=TRIGGER_PCT, line_dash="dash",
              annotation_text=f"trigger {TRIGGER_PCT:.3%}",
              annotation_position="top left", row="all", col=1)

# ── Row-2: maker limits are BUYS (edges expressed vs. BUY price) ──
fig.add_scatter(x=df.index, y=df["edge_BB"],
                mode="lines", name="BBSH  (Buy BNC / Sell HTX)",
                line=dict(width=1), legendgroup="row2", row=2, col=1)

fig.add_scatter(x=df.index, y=df["edge_HS"],
                mode="lines", name="BHSB  (Buy HTX / Sell BNC)",
                line=dict(width=1), legendgroup="row2", row=2, col=1)

# ── cosmetic tweaks ───────────────────────────────────────────────
fig.update_yaxes(title_text="% after fees", row=1, col=1)
fig.update_yaxes(title_text="% after fees", row=2, col=1)
fig.update_layout(
    title="Net Edge (%) — four legs, separated by maker-side",
    hovermode="x unified",
    height=650,                       # a bit taller for readability
    legend=dict(borderwidth=0))

fig.show()


In [6]:
import pandas as pd, plotly.graph_objects as go

def run_leg(tag,
            maker_rate, taker_rate,
            usd_col, q_col, v_m_col, v_t_col, edge_col,
            maker_is_sell=True):
    """
    maker_is_sell=True   → maker LIMIT is a SELL (cash in), taker is BUY (cash out)
    maker_is_sell=False  → maker LIMIT is a BUY  (cash out), taker is SELL (cash in)
    """
    trades, nav = [], []
    inv = xfers = pnl = 0.0

    for ts, row in df.iterrows():

        if row[edge_col] <= TRIGGER_PCT or row[q_col] == 0:
            nav.append({"ts": ts, "nav": pnl})
            continue

        # cash-flow orientation ----------------------------------
        if maker_is_sell:
            proceeds = row[usd_col] * (1 - maker_rate)                 # maker cash-in
            cost     = row[q_col] * row[v_t_col] * (1 + taker_rate)    # taker cash-out
        else:
            proceeds = row[q_col] * row[v_t_col] * (1 - taker_rate)    # taker cash-in
            cost     = row[usd_col] * (1 + maker_rate)                 # maker cash-out

        # inventory & batch-transfer ------------------------------
        inv += row[q_col]               # LINK accrues on the BUY venue
        x_cost = 0.0
        while inv >= TRANSFER_TH_LINK:
            x_cost += WITHDRAW_FEE_LNK * row[v_t_col]
            inv    -= TRANSFER_TH_LINK
            xfers  += 1

        trade_pnl = proceeds - cost - x_cost
        pnl      += trade_pnl

        trades.append(dict(ts=ts, edge_pct=row[edge_col], qty=row[q_col],
                           maker_px=row[v_m_col], taker_px=row[v_t_col],
                           proceeds=proceeds, cost=cost,
                           transfer=x_cost, inv_after=inv,
                           pnl=trade_pnl, cum_pnl=pnl))
        nav.append({"ts": ts, "nav": pnl})

    trades_df = pd.DataFrame(trades)
    nav_df    = pd.DataFrame(nav).set_index("ts")
    trades_df.to_csv(f"trade_log_{tag}.csv", index=False)

    fig = go.Figure(go.Scatter(x=nav_df.index, y=nav_df["nav"], mode="lines"))
    fig.update_layout(title=f"{tag} – Cumulative P&L", yaxis_title="USDT")
    fig.show()

    print(f"{tag}: trades={len(trades_df)}  transfers={xfers}  "
          f"inventory={inv:.1f} LINK  P&L={pnl:.2f} USDT")
    return trades_df


# ─── run all four legs ────────────────────────────────────────────
trades_SHBB = run_leg("SHBB",
    MAKER_HTX_BP/10_000, TAKER_BNC_BP/10_000,
    "USD_SH", "Q_SH", "VWAP_SH_M", "VWAP_SH_T", "edge_SH",
    maker_is_sell=True)           # SELL Huobi, BUY Binance

trades_BBSH = run_leg("BBSH",
    MAKER_BNC_BP/10_000, TAKER_HTX_BP/10_000,
    "USD_BB", "Q_BB", "VWAP_BB_M", "VWAP_BB_T", "edge_BB",
    maker_is_sell=False)          # BUY Binance, SELL Huobi

trades_SBBH = run_leg("SBBH",
    MAKER_BNC_BP/10_000, TAKER_HTX_BP/10_000,
    "USD_SB", "Q_SB", "VWAP_SB_M", "VWAP_SB_T", "edge_SB",
    maker_is_sell=True)           # SELL Binance, BUY Huobi

trades_BHSB = run_leg("BHSB",
    MAKER_HTX_BP/10_000, TAKER_BNC_BP/10_000,
    "USD_HS", "Q_HS", "VWAP_HS_M", "VWAP_HS_T", "edge_HS",
    maker_is_sell=False)          # BUY Huobi, SELL Binance


SHBB: trades=6  transfers=0.0  inventory=2960.0 LINK  P&L=4.90 USDT


BBSH: trades=12  transfers=1.0  inventory=7996.7 LINK  P&L=86.71 USDT


SBBH: trades=73  transfers=9.0  inventory=6145.2 LINK  P&L=567.88 USDT


BHSB: trades=10  transfers=0.0  inventory=4240.0 LINK  P&L=11.58 USDT


In [7]:
trades_BBSH


,ts,edge_pct,qty,maker_px,taker_px,proceeds,cost,transfer,inv_after,pnl,cum_pnl
0,2021-08-20 00:10:13,0.006696,1145.522,26.791830,26.807239,30698.516389,30696.461337,0.00000,1145.522,2.055053,2.055053
1,2021-08-20 00:19:48,0.112661,1188.202,26.971620,27.015722,32089.926810,32053.821585,0.00000,2333.724,36.105225,38.160278
2,2021-08-20 00:24:17,0.007923,1751.674,27.180628,27.196594,47624.417970,47620.645932,0.00000,4085.398,3.772038,41.932316
3,2021-08-20 00:41:39,0.003827,1171.095,27.311138,27.326062,31991.237614,31990.013668,0.00000,5256.493,1.223946,43.156263
4,2021-08-20 00:41:40,0.020244,1823.623,27.316142,27.335555,49833.894628,49823.810106,0.00000,7080.116,10.084522,53.240785
5,2021-08-20 00:49:08,0.044581,1189.292,27.295639,27.321682,32483.125271,32468.653262,0.00000,8269.408,14.472009,67.712794
6,2021-08-20 01:19:06,0.009173,1409.642,27.099742,27.116000,38211.697287,38208.192968,0.00000,9679.050,3.504319,71.217113
7,2021-08-20 01:19:08,0.009344,1645.904,27.099696,27.116000,44616.140418,44611.972502,2.98276,1324.954,1.185157,72.402270
8,2021-08-20 01:19:09,0.003072,1209.163,27.101395,27.116000,32777.237431,32776.230607,0.00000,2534.117,1.006824,73.409094
9,2021-08-20 01:19:26,0.005069,1539.287,27.100854,27.116000,41726.033193,41723.918492,0.00000,4073.404,2.114701,75.523795


In [8]:
trades_SHBB

,ts,edge_pct,qty,maker_px,taker_px,proceeds,cost,transfer,inv_after,pnl,cum_pnl
0,2021-08-20 00:02:46,0.005907,674.399779,27.057717,27.042487,18243.630862,18242.553555,0.0,674.399779,1.077307,1.077307
1,2021-08-20 00:02:59,0.009059,202.480000,27.078877,27.062782,5481.702789,5481.206394,0.0,876.879779,0.496396,1.573703
2,2021-08-20 00:13:01,0.010324,469.430000,27.040563,27.024150,12690.808287,12689.498593,0.0,1346.309779,1.309694,2.883397
3,2021-08-20 00:15:21,0.002881,461.990000,27.014389,27.000000,12477.581962,12477.222644,0.0,1808.299779,0.359318,3.242715
4,2021-08-20 00:26:19,0.004062,700.910000,27.244833,27.230000,19091.898531,19091.123318,0.0,2509.209779,0.775212,4.017928
5,2021-08-20 00:49:18,0.007133,450.750000,27.342666,27.326940,12321.945846,12321.067199,0.0,2959.959779,0.878647,4.896575


In [9]:
trades_SBBH

,ts,edge_pct,qty,maker_px,taker_px,proceeds,cost,transfer,inv_after,pnl,cum_pnl
0,2021-08-20 00:00:17,0.008083,1180.784,27.130653,27.114684,32029.353737,32026.765932,0.000,1180.784,2.587806,2.587806
1,2021-08-20 00:00:18,0.004765,709.473,27.129378,27.114309,19243.903906,19242.987185,0.000,1890.257,0.916722,3.504527
2,2021-08-20 00:00:20,0.020924,1007.658,27.138567,27.119113,27341.198746,27335.480847,0.000,2897.915,5.717899,9.222426
3,2021-08-20 00:00:22,0.014714,1379.011,27.146898,27.129121,37428.758401,37423.253742,0.000,4276.926,5.504660,14.727086
4,2021-08-20 00:00:24,0.008901,1289.220,27.143897,27.127699,34987.806519,34984.693448,0.000,5566.146,3.113072,17.840158
...,...,...,...,...,...,...,...,...,...,...,...
68,2021-08-20 00:43:40,0.026488,2494.296,27.421181,27.400000,68383.546767,68365.443700,0.000,8538.450,18.103067,515.435162
69,2021-08-20 00:43:41,0.029987,2470.899,27.422140,27.400000,67744.464222,67724.162037,3.014,1009.349,17.288185,532.723347
70,2021-08-20 00:43:42,0.037812,2445.493,27.424284,27.400000,67053.152461,67027.816270,0.000,3454.842,25.336191,558.059538
71,2021-08-20 00:43:43,0.019473,1741.255,27.419259,27.400000,47734.849635,47725.558903,0.000,5196.097,9.290732,567.350270


In [10]:
trades_BHSB

,ts,edge_pct,qty,maker_px,taker_px,proceeds,cost,transfer,inv_after,pnl,cum_pnl
0,2021-08-20 00:01:02,0.009823,548.23,27.024349,27.040628,14820.332892,14818.877628,0.0,548.23,1.455264,1.455264
1,2021-08-20 00:01:21,0.006762,722.55,27.025241,27.040694,19532.782553,19531.462171,0.0,1270.78,1.320382,2.775646
2,2021-08-20 00:01:53,0.030564,435.23,26.964841,26.986679,11742.123511,11738.536605,0.0,1706.01,3.586906,6.362552
3,2021-08-20 00:02:16,0.003970,367.84,26.921346,26.935987,9905.359224,9904.966088,0.0,2073.85,0.393136,6.755688
4,2021-08-20 00:03:34,0.010101,566.37,27.088212,27.104605,15346.936619,15345.386946,0.0,2640.22,1.549674,8.305362
5,2021-08-20 00:04:17,0.015783,232.14,26.995267,27.013139,6269.074144,6268.085092,0.0,2872.36,0.989053,9.294415
6,2021-08-20 00:04:26,0.006052,283.54,26.982478,26.997714,7652.788556,7652.325520,0.0,3155.90,0.463036,9.757451
7,2021-08-20 00:04:57,0.008159,447.73,26.928513,26.944286,12060.387301,12059.403637,0.0,3603.63,0.983664,10.741115
8,2021-08-20 00:05:11,0.011363,168.90,26.969338,26.986000,4556.659178,4556.141585,0.0,3772.53,0.517593,11.258708
9,2021-08-20 00:45:15,0.002489,467.48,27.321709,27.336163,12775.531368,12775.213471,0.0,4240.01,0.317897,11.576605
